In [3]:
def xmile2devsml(archivoXmile, archivoDevsml):
    import xml.etree.ElementTree as ET
    from jinja2 import DictLoader, Environment, FileSystemLoader
    import os
    import re
    import numpy as np

    # Parametros para funciones generadoras
    PATH = './'
    TEMPLATE_ENVIRONMENT = Environment(
        autoescape=False,
        loader=FileSystemLoader(os.path.join(PATH, 'templates')),
        trim_blocks=False)
    TEMPLATE_DEVSML = 'template-devsml.xml'

    def render_template(template_filename, context):
        return TEMPLATE_ENVIRONMENT.get_template(template_filename).render(context)

    tree = ET.parse(archivoXmile)
    root = tree.getroot()

    model = root.find('model')
    
    variables = model.find('variables')

    flows  = variables.findall('flow')
    auxs   = variables.findall('aux')
    stocks = variables.findall('stock')

    flows_, auxs_, ctes_, stocks_ = {}, {}, {}, {}
    # Obtengo inflows y Outflows de cada Stock
    # Obtengo origen y destino de cada Flow
    for stock in stocks:
        stockName = stock.get('name').replace('"', '')
        initialValue = stock.find('eqn').text
        stocks_[stockName] = {'inflows' : [], 'outflows' : [], 'initialValue' : initialValue}
        if stock.find('inflow') is not None:
            for inflow in stock.findall('inflow'):
                stocks_[stockName]['inflows'].append(inflow.text)
                if inflow.text not in flows_.keys():
                    flows_[inflow.text.replace('"', '')] = { 'orig' : None, 'dest' : stockName }
                else:
                    flows_[inflow.text.replace('"', '')]['dest'] = stockName
        if stock.find('outflow') is not None:
            for outflow in stock.findall('outflow'):
                stocks_[stockName]['outflows'].append(outflow.text)
                if outflow.text not in flows_.keys():
                    flows_[outflow.text.replace('"', '')] = { 'orig' : stockName, 'dest' : None }
                else:
                    flows_[outflow.text.replace('"', '')]['orig'] = stockName

    for aux in auxs:
        auxName = aux.get('name').replace('"', '')
        assert(aux.find('eqn') != None)

        if(re.search('[a-zA-Z]', aux.find('eqn').text) is None):
            # Cte's
            ctes_[auxName] = { 'eqn' : aux.find('eqn').text , 'units' : None }
            if aux.find('units') is not None:
                ctes_[auxName]['units'] = aux.find('units').text
        else:
            # Aux's
            auxs_[auxName] = { 'eqn' : aux.find('eqn').text , 'units' : None }
            if aux.find('units') is not None:
                auxs_[auxName]['units'] = aux.find('units').text

    ###
    # Completo informacion sobre cada Flow en 'flows_'
    for flow in flows:
        flowName = flow.get('name').replace('"', '')
        flows_[flowName]['eqn'] = flow.find('eqn').text
        flows_[flowName]['units'] = None
        if flow.find('units') is not None:
            flows_[flowName]['units'] = flow.find('units').text
        
        text = flow.find('eqn').text.replace(' ', '')
        #print(re.split("[()-/*]+", text))
        flows_[flowName]['params_eqn'] = filter(lambda x : x != '', re.split("[()-/*]+", text))

    for stockName, attr in stocks_.iteritems():
        for i, flow in enumerate(attr['outflows']):
            val = stocks_[stockName]['outflows'][i].replace('"', '')
            stocks_[stockName]['outflows'][i] = (val, flows_[val]['dest'])
        for i, flow in enumerate(attr['inflows']):
            val = stocks_[stockName]['inflows'][i]
            stocks_[stockName]['inflows'][i] = (val, flows_[val]['orig'])

    print '* Stocks: ', stocks_
    print '* Flows: ', flows_
    print '* Auxs: ', auxs_
    print '* Ctes: ', ctes_
    # TODO : mejorar los puertos para Ftot's
    
    context = {
        'stocks_' : stocks_,
        'flows_' : flows_,
        'auxs_' : auxs_,
        'ctes_' : ctes_
    }
    # Guardar el .devsml en output
    with open(archivoDevsml, 'w') as f:
        f.write(render_template(TEMPLATE_DEVSML,context))

In [7]:
#archivoXmile  = 'modelos traducidos/SIR.xmile'
#archivoDevsml = 'sir/sir-devsml-traduccion.xml'
#xmile2devsml(archivoXmile, archivoDevsml)

#archivoXmile  = 'modelos traducidos/teacup.xmile'
#archivoDevsml = 'teacup/teacup-devsml-traduccion.xml'
#xmile2devsml(archivoXmile, archivoDevsml)

#archivoXmile = 'modelos traducidos/stock_to_capacity.xmile'
#archivoDevsml = 'stock_to_capacity/stock_to_capacity-devsml-traduccion.xml'
#xmile2devsml(archivoXmile, archivoDevsml)

#print '#########'

#archivoXmile = 'modelos traducidos/samuelson.xmile'
#archivoDevsml = 'samuelson/samuelson-devsml-traduccion.xml'
#xmile2devsml(archivoXmile, archivoDevsml)

#archivoXmile = 'modelos traducidos/testModel.xmile'
#archivoDevsml = 'testModel/testModel-devsml-traduccion.xml'
#xmile2devsml(archivoXmile, archivoDevsml)

#archivoXmile = 'modelos traducidos/testModel2.xmile'
#archivoDevsml = 'testModel2/testModel2-devsml-traduccion.xml'
#xmile2devsml(archivoXmile, archivoDevsml)

archivoXmile = 'goodwin/goodwin.xmile'
archivoDevsml = 'goodwin/goodwin-devsml-traduccion.xml'
xmile2devsml(archivoXmile, archivoDevsml)

#archivoXmile = 'lotka-volterra/lotka-volterra-2.xmile'
#archivoDevsml = 'lotka-volterra/lotka-volterra-devsml-traduccion.xml'
#xmile2devsml(archivoXmile, archivoDevsml)

* Stocks:  {'LaborProductivity': {'initialValue': '1', 'outflows': [], 'inflows': [('chgLaborProductivity', None)]}, 'WageRate': {'initialValue': '0.8', 'outflows': [], 'inflows': [('chgWageRate', None)]}, 'Debt': {'initialValue': '0', 'outflows': [], 'inflows': [('chgDebt', None)]}, 'Population': {'initialValue': '150', 'outflows': [], 'inflows': [('chgPopulation', None)]}, 'Capital': {'initialValue': '300', 'outflows': [], 'inflows': [('chgCapital', None)]}}
* Flows:  {'chgPopulation': {'dest': 'Population', 'units': None, 'eqn': 'Betaa * Population', 'params_eqn': ['Betaa', 'Population'], 'orig': None}, 'chgLaborProductivity': {'dest': 'LaborProductivity', 'units': None, 'eqn': 'Alphaa * LaborProductivity', 'params_eqn': ['Alphaa', 'LaborProductivity'], 'orig': None}, 'chgDebt': {'dest': 'Debt', 'units': None, 'eqn': 'InvestmentGross', 'params_eqn': ['InvestmentGross'], 'orig': None}, 'chgWageRate': {'dest': 'WageRate', 'units': None, 'eqn': 'wageFunction * WageRate', 'params_eqn': 